In [1]:
import numpy as np
import pandas as pd
from DHRobotGT import myCobot320
from scipy.spatial.transform import Rotation as R

# Comparación entre el modelo del robot y el de *roboticstoolbox*

Vamos a comparar los datos que surgen de: mandar al robot a una $\overline{q}$ aleatoria, preguntarle a qué $\overline{q}$ llegó, mandarlo al lugar que acaba de responder y preguntarle en qué pose final se encuentra. Nos va a interesar el $\overline{q}$ que responde cuando le preguntamos por primera vez y la pose final que nos devuelve luego de haberlo mandado a estos ángulos.

In [2]:
cobot = myCobot320()

El cobot define las poses como $[p_x, p_y, p_z, r_x, r_y, r_z]$ así que vamos a corroborarlo con una función que toma un $\overline{q}$, resuelve el problema directo y devuelve los ángulos de las rotaciones que corresponderían. El resultado debería coincidir con el dato que nos devolvió el robot en el ensayo.

In [3]:
def cheq_rot(q):
    pose = cobot.fkine(np.radians(q))
    rot = R.from_matrix(pose.R)
    ang = rot.as_euler('xyz', degrees=True)
    return ang

Leemos los datos del txt generado en el ensayo y armamos una array de $[N\times2\times6]$ que tiene el $\overline{q}$ y la pose para cada una de las $N$ iteraciones realizadas.

In [4]:
import ast

def leer_datos_txt(path_txt):
    qs = []
    poses = []

    with open(path_txt, 'r', encoding='utf-8') as f:
        lineas = f.readlines()

    for linea in lineas[2:]:
        linea = linea.strip()
        if not linea:
            continue

        partes = linea.split(';')
        if len(partes) != 3:
            continue

        q_str = partes[1].strip()
        pose_str = partes[2].strip()

        try:
            q = np.array(ast.literal_eval(q_str), dtype=float)
            pose = np.array(ast.literal_eval(pose_str), dtype=float)
            qs.append(q)
            poses.append(pose)
        except Exception as e:
            print(f"Error al procesar línea: {linea}\n{e}")

    # Stack para formar arrays de shape (N, 6)
    qs_array = np.stack(qs)
    poses_array = np.stack(poses)

    # Concatenar en el eje 1: (N, 2, 6)
    return np.stack((qs_array, poses_array), axis=1)


In [5]:
ruta = 'QvsPose24-04.txt'
resultados = leer_datos_txt(ruta)

Verificamos cómo interpreta el archivo:

In [6]:
print(f'Dimensiones: {resultados.shape} \n \nPrimeras 3 filas:\n{resultados[:3]}')

Dimensiones: (43, 2, 6) 
 
Primeras 3 filas:
[[[  36.29   79.8   -78.04    4.13   14.58   82.61]
  [ -14.5  -199.5   413.9   -48.64   87.75  -87.84]]

 [[ -43.24  -25.66   81.73   92.9   -45.     71.27]
  [-129.2   -63.8   257.2   120.43  -43.99  -24.85]]

 [[  53.87  -77.25   62.66   34.01  -28.56  -91.4 ]
  [ 177.5    -4.8   398.9  -120.6   -71.79 -123.97]]]


Agregamos una nueva columna donde se aplica la función `cheq_rot` que se mencionaba antes. La idea es ver que coincidan con lo que devuelve el cobot.

In [7]:
resultados_con_rot = []

for q, pose in resultados:
    rot = cheq_rot(q)
    fila = [q, pose, rot]  # Une los tres arrays en uno solo
    resultados_con_rot.append(fila)

# Convertimos a un nuevo ndarray de objetos
resultados_con_rot = np.array(resultados_con_rot, dtype=object)

Miramos los primeros 3 casos:

In [8]:
for i in range(len(resultados_con_rot[:3])):
    print(f'Coordenadas cobot: {resultados_con_rot[i, 1][3:]} \nCoordenadas rotacion: {resultados_con_rot[i, 2]} \n')

Coordenadas cobot: [-48.64  87.75 -87.84] 
Coordenadas rotacion: [-48.74796994  87.7547324   92.02650092] 

Coordenadas cobot: [120.43 -43.99 -24.85] 
Coordenadas rotacion: [120.43723643 -43.98520638 155.15519659] 

Coordenadas cobot: [-120.6   -71.79 -123.97] 
Coordenadas rotacion: [-120.58863884  -71.7978083    56.01296211] 



Se ve que hay una diferencia de $180^\circ$ en $r_z$. Las demás rotaciones, salvo diferencias en el valor, son coherentes. En un dataframe se ve todo más prolijo.

In [9]:
cobot_coords = []
rot_coords = []
dif_coords = []
np.set_printoptions(suppress=True, precision=2)

# Extraer los valores deseados
for i in range(len(resultados_con_rot)):
    robot_real = resultados_con_rot[i, 1][3:]
    rot = resultados_con_rot[i, 2]
    dif = np.abs(robot_real - rot)
    cobot_coords.append(robot_real)
    rot_coords.append(rot)
    dif_coords.append(dif)

df = pd.DataFrame({
    'Cobot': cobot_coords,
    'Rotación': rot_coords,
    'Diferencia': dif_coords
})
# Mostrar el DataFrame con celdas centradas
df.style.set_properties(**{
    'text-align': 'center'
}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}]).format({
    'Chequeo Rz': '{:,.2f}',
    # 'Diferencia': '{:.2f}'
})

,Cobot,Rotación,Diferencia
0,[-48.64 87.75 -87.84],[-48.75 87.75 92.03],[ 0.11 0. 179.87]
1,[120.43 -43.99 -24.85],[120.44 -43.99 155.16],[ 0.01 0. 180.01]
2,[-120.6 -71.79 -123.97],[-120.59 -71.8 56.01],[ 0.01 0.01 179.98]
3,[-124.01 -6.44 -176.74],[-124.02 -6.45 3.25],[ 0.01 0.01 179.99]
4,[-90. 0.7 179.64],[-90. 0.61 -0.27],[ 0. 0.09 179.91]
5,[-89.99 0.61 179.82],[-90. 0.61 -0.18],[ 0.01 0. 180. ]
6,[-90. 0.52 179.73],[-90. 0.7 -0.34],[ 0. 0.18 180.07]
7,[-90. 0.79 179.64],[-90. 0.79 -0.34],[ 0. 0. 179.98]
8,[-90. 0.7 179.64],[-90. 0.88 -0.34],[ 0. 0.18 179.98]
9,[-90. 0.61 179.64],[-90. 0.61 -0.34],[ 0. 0. 179.98]


Efectivamente la diferencia es de $180^\circ$ para $r_z$. Esto no es algo trivial si no que trae consecuencias cuando se resuelve el problema directo. Podemos chequearlo transformando la pose que devuelve el robot en coordenadas y ángulos de Euler a su matriz de rototraslación. Esto es sencillo con la librería *spatialmath*.

In [10]:
from spatialmath import SE3

def pose_to_matrix(pose):
    x, y, z, rx, ry, rz = pose

    # Pasamos a radianes
    rx, ry, rz = np.deg2rad([rx, ry, rz])

    # Crear objeto SE3 con rotación en RPY (Z-Y-X por defecto) y traslación
    T = SE3(x, y, z) * SE3.RPY([rx, ry, rz], order='zyx')
    T_matrix = T.A

    return T_matrix

Ahora sí, vemos un ejemplo del inconveniente en la rotación $r_z$ para el primer valor de $\overline{q}$:

In [11]:
print(f'Pose obtenida con el problema directo:\n{cobot.fkine(np.radians(resultados[0, 0]))}\n\
Pose obtenida por el robot real:\n{pose_to_matrix(resultados[0, 1])}')

Pose obtenida con el problema directo:
  -0.001385 -0.6324   -0.7746    14.64     
   0.03915  -0.7741    0.6319    199.5     
  -0.9992   -0.02945   0.02583   413.9     
   0         0         0         1         

Pose obtenida por el robot real:
[[   0.      0.63    0.77  -14.5 ]
 [  -0.04    0.77   -0.63 -199.5 ]
 [  -1.     -0.03    0.03  413.9 ]
 [   0.      0.      0.      1.  ]]


Además de haber signos intercambiados en las rotaciones, se invierten también $p_x$ y $p_y$. Esto se soluciona rotando la terna base del robot que modelamos con la toolbox.

In [12]:
cobot.base = SE3.Rz(np.pi)

Chequeamos nuevamente:

In [13]:
print(f'Pose obtenida con el problema directo:\n{cobot.fkine(np.radians(resultados[0, 0]))}\n\
Pose obtenida por el robot real:\n{pose_to_matrix(resultados[0, 1])}')

Pose obtenida con el problema directo:
   0.001385  0.6324    0.7746   -14.64     
  -0.03915   0.7741   -0.6319   -199.5     
  -0.9992   -0.02945   0.02583   413.9     
   0         0         0         1         

Pose obtenida por el robot real:
[[   0.      0.63    0.77  -14.5 ]
 [  -0.04    0.77   -0.63 -199.5 ]
 [  -1.     -0.03    0.03  413.9 ]
 [   0.      0.      0.      1.  ]]


Vemos que ya el modelo ajusta sus referencias a lo mismo que interpreta el robot real. Ya podemos proceder a ajustar los parámetros DH.

## Optimizador de parámetros

Ahora vamos a ver qué combinación de parámetros DH se ajusta mejor a las poses que alcanzó el robot real, a modo de calibrar nuestro modelo de *roboticstoolbox*.

Para las $\overline{q}$ no hace falta hacer nada más que indexar lo que se lee del  archivo txt y para las poses usamos la función `pose_to_matrix` que mencionamos arriba.

In [14]:
poses_lista = []
for pose in resultados[:, 1]:
    poses_lista.append(pose_to_matrix(pose))
    
poses_reales = np.array(poses_lista)
qs_reales = resultados[:, 0]

En el optimizador ya tenemos los datos del robot real, la generación del dataset es a fines demostrativos.

In [15]:
# Identificación de parámetros DH simulada en roboticstoolbox-python
from roboticstoolbox import DHRobot, RevoluteDH
from scipy.optimize import minimize

# Crear el robot
def create_robot(param=np.array([0,135,120,0,0,0,173.9,0,0,88.78,95,65.5]),robot_name='real_bot'):
    return DHRobot([
        RevoluteDH(alpha=-np.pi/2,a=param[0],d=param[6],offset=0,qlim=[-170*np.pi/180,170*np.pi/180]),
        RevoluteDH(alpha=0,a=param[1],d=param[7],offset=-np.pi/2,qlim=[-120*np.pi/180,120*np.pi/180]),
        RevoluteDH(alpha=0,a=param[2],d=param[8],offset=0,qlim=[-148*np.pi/180,148*np.pi/180]),
        RevoluteDH(alpha=np.pi/2,a=param[3],d=param[9],offset=np.pi/2,qlim=[-120*np.pi/180,135*np.pi/180]),
        RevoluteDH(alpha=-np.pi/2,a=param[4],d=param[10],offset=0,qlim=[-169*np.pi/180,169*np.pi/180]),
        RevoluteDH(alpha=0,a=param[5],d=param[11],offset=0,qlim=[-180*np.pi/180,180*np.pi/180])
        ], name=robot_name)

# Generar dataset sintético (q, pose)
# def generate_dataset(robot, n_samples=100):
#     qs, poses = [], []
#     for _ in range(n_samples):
#         q = np.random.uniform(low=-np.pi, high=np.pi, size=(6,))
#         T = robot.fkine(q).A
#         qs.append(q)
#         poses.append(T)
#     return np.array(qs), np.array(poses)

# Funcón de error entre pose estimada y real (sólo traslación)
def pose_error(params, qs, poses, flag_disp=True):

    robot = create_robot(params,'model_bot')
    robot.base = SE3.Rz(np.pi)

    total_error = 0.0
    for q, T_true in zip(qs, poses):
        T_pred = robot.fkine(q).A
        # Error de posición: distancia euclidea
        err_pos = np.linalg.norm(T_true[:3, 3] - T_pred[:3, 3])

        # Error de orientación: ángulo entre rotaciones
        R_true = T_true[:3, :3]
        R_pred = T_pred[:3, :3]
        R_delta = R_true.T @ R_pred  # rotación relativa
        theta_err = np.arccos(np.clip((np.trace(R_delta) - 1) / 2, -1.0, 1.0))  # ángulo de rotación

        # Pesos de los errores
        w_pos = 1
        w_ori = 0    # Desactivo la parte de orientación como sospecho que solo tengo incertezas en las longitudes
        total_error += (w_pos * err_pos**2 + w_ori * theta_err**2)

    loss = total_error / len(qs)
    if flag_disp:
        print(f"Loss= {loss}") 
    return loss

# Experimento principal
if __name__ == '__main__':
    x_real = [0,135,120,0,0,0,173.9,0,0,88.78,95,65.5]
    x_pert = [0,140,110,0,0,0,176,0,0,95,80,60]

    # Crear el robot "real" (modelo verdadero)
    real_robot = create_robot(x_real)
    real_robot.base = SE3.Rz(np.pi)

    # qs, poses = generate_dataset(real_robot, n_samples=200)
    qs, poses = np.radians(qs_reales), poses_reales
    
    
    # Ajusto y arranco el optimizador
    def callback(xk):
        print("****** Iteracion\n")
        for i in range(6):
            print(f"a{i}={xk[i]:.4f}  real={x_real[i]:.4f}, d1={xk[i+6]:.4f}  real={x_real[i+6]:.4f}")
        
    optim_options = {
        'maxiter': 1000,      # Número máximo de iteraciones
        'maxfev': 2000,       # Número máximo de evaluaciones de la función objetivo
        'xatol': 1e-2,        # Tolerancia en los parámetros (x)
        'fatol': 1e-2,        # Tolerancia en el valor de la función (f(x))
        'disp': True          # Mostrar progreso
    }        
    res = minimize(pose_error, x_pert, args=(qs, poses), method='Nelder-Mead', callback=None, options=optim_options)

    # Muestro resultados
    print("\n*********************************************************************************\nParámetros estimados vs reales:")
    print(f"{'Link':<5} {'a_est':>8} {'a_real':>10} {'a_ini':>10} | {'d_est':>8} {'d_real':>10} {'d_ini':>10}")
    print("-" * 65)
    for i in range(6):
        print(f"{i+1:<5} "
            f"{res.x[i]:>8.2f} {x_real[i]:>10.2f} {x_pert[i]:>10.2f} | "
            f"{res.x[i+6]:>8.2f} {x_real[i+6]:>10.2f} {x_pert[i+6]:>10.2f}")

    # Comparación de error antes y después
    err_initial = pose_error(x_pert, qs, poses,False)
    err_final = pose_error(res.x, qs, poses,False)
    print(f"\n\nError inicial: {err_initial:.6f}")
    print(f"Error final:   {err_final:.6f}")

Loss= 423.8375361620156
Loss= 423.8373777498704
Loss= 382.481486872421
Loss= 280.2358585997923
Loss= 423.83821625481886
Loss= 423.8381604813994
Loss= 423.8392565359164
Loss= 361.8926778757239
Loss= 423.8394329951259
Loss= 423.8394329951257
Loss= 482.438677754386
Loss= 288.7544294383692
Loss= 423.43433160073823
Loss= 326.7204986623078
Loss= 325.38346454944036
Loss= 312.32549519684466
Loss= 298.3014094673544
Loss= 283.5885704782328
Loss= 268.66422394431225
Loss= 235.44373971315943
Loss= 248.35725177723847
Loss= 237.08349144793067
Loss= 247.73250405909346
Loss= 200.0931202291733
Loss= 168.6705971450601
Loss= 185.0026650809874
Loss= 205.0175833900448
Loss= 167.60315113402848
Loss= 142.20558204533558
Loss= 150.85402674260374
Loss= 139.73242360272968
Loss= 122.58281925328104
Loss= 217.80168087387057
Loss= 139.6304137449375
Loss= 213.5970895224278
Loss= 132.9901613149292
Loss= 93.56212699851616
Loss= 78.37655475437526
Loss= 99.16793332157124
Loss= 98.83241217348687
Loss= 55.77173082486067
Los

Probamos los parámetros ajustados comparando con los que teníamos:

In [16]:
# DH_ajustados = [0,137.59,122.01,0,0,0,164.72,0,0,88.67,96.64,65.86]
DH_ajustados = res.x
# DH_ajustados = [0,137.42,122.34,0.88,0,0,164.87,0,0,90.26,96.42,65.87]
robot_ajustado = create_robot(DH_ajustados)
robot_original = create_robot(x_real)
robot_original.base = SE3.Rz(np.pi)
robot_ajustado.base = SE3.Rz(np.pi)

it = 6
print(f'Pose obtenida con parámetros ajustados:\n{robot_ajustado.fkine(qs[it])}\n\
Pose obtenida con parámetros iniciales:\n{robot_original.fkine(qs[it])}\n\
Pose obtenida por el robot real:\n{pose_to_matrix(resultados[it, 1])}')

Pose obtenida con parámetros ajustados:
  -0.9999    0.01222  -0.005934 -3.622     
   0.005934 -6.177e-05 -1        -154.2     
  -0.01222  -0.9999   -1.072e-05  523.8     
   0         0         0         1         

Pose obtenida con parámetros iniciales:
  -0.9999    0.01222  -0.005934 -3.623     
   0.005934 -6.177e-05 -1        -154.3     
  -0.01222  -0.9999   -1.072e-05  523.9     
   0         0         0         1         

Pose obtenida por el robot real:
[[  -1.      0.01   -0.     -2.8 ]
 [   0.     -0.     -1.   -154.2 ]
 [  -0.01   -1.      0.    523.8 ]
 [   0.      0.      0.      1.  ]]


Vemos que mejorá notablemente cada una de las coordenadas del vector de traslación, igualmente la mejora va a depender de la pose analizada. 

Claro que los parámetros no afectan a la rotación, ahí el error tiene que ver con la lectura de los ángulos o con algo relacionado a las juntas. No obstante, podemos decir que el modelo es notablemente más preciso respecto al cobot del laboratorio.

# Matriz a pose

Para poder usar una pose calculada como matriz 4x4 en *roboticstoolbox* y enviarle las coordenadas al cobot mediante `send_coords` tenemos que convertir dicha matriz al formato que entiende el robot: $[p_x, p_y, p_z, r_x, r_y, r_z]$. 

Lo que se gana con esta metodología es no tener que recurrir a la cinemática inversa de la toolbox y la posibilidad de pedirle que vaya de una pose a otra sin tener que mirar los ángulos. Esto es útil para hacer que pase cerca de singularidades moviéndose en línea recta.

In [17]:
def matrix_to_pose(T_matrix):
 
    T = SE3(T_matrix)

    # Extraer traslación
    x, y, z = T.t

    # Extraer rotación como RPY (en radianes), orden ZYX
    rx, ry, rz = T.rpy(order='zyx', unit='rad')

    # Convertimos a grados
    rx, ry, rz = np.rad2deg([rx, ry, rz])

    return [x, y, z, rx, ry, rz]


Chequeamos que funcione:

In [18]:
coord_prueba = [-106.7, -255.1, 430.5, -92.8, 78.11, -149.75]
matriz_prueba = pose_to_matrix(coord_prueba)
coord_cheq = matrix_to_pose(matriz_prueba)

print(f'Coordenadas de prueba: {coord_prueba}\nMatriz obtenida:\n{matriz_prueba}\nCoordenadas obtenidas: {coord_cheq}')
type(coord_cheq)

Coordenadas de prueba: [-106.7, -255.1, 430.5, -92.8, 78.11, -149.75]
Matriz obtenida:
[[  -0.18    0.82    0.54 -106.7 ]
 [  -0.1     0.53   -0.84 -255.1 ]
 [  -0.98   -0.21   -0.01  430.5 ]
 [   0.      0.      0.      1.  ]]
Coordenadas obtenidas: [-106.7, -255.1, 430.5, -92.8, 78.11, -149.75]


list